In [ ]:
%pip install -r requirements.txt # Only run if haven't run before

### Step 1 - Import dependencies

In [101]:
import psycopg2 # Library for interacting with S3 protocol
from psycopg2 import sql # Library for loading in yaml files
import yaml # Library for loading in yaml files

### Step 2 - Load Secrets & Global Variables

In [102]:
keys_file_path = 'keys.yaml' # TODO: Make sure this is in your repository and get your own access/secret keys from MinIO!

with open(keys_file_path, 'r') as file:
    keys = yaml.safe_load(file)

postgres_credentials = keys.get('postgres', {})
postgres_endpoint = keys.get('services')['postgres_endpoint'] 

In [103]:
username = postgres_credentials.get('username')
password = postgres_credentials.get('password')

### Step 3 - Set up postgres session

In [104]:
dbname = "test"

In [105]:
db_params = {
    'dbname': dbname,
    'user': username,
    'password': password,
    'host': postgres_endpoint,
    'port': 5432 # NOTE: Postgres default port is 5432
}

### Step 4 - Profit

In [106]:
# Function to execute a SQL query
def execute_query(query, params=None):
    try:
        connection = psycopg2.connect(**db_params)
        cursor = connection.cursor()

        # If the query has parameters, use sql.SQL and sql.Identifier to safely construct the query
        if params:
            cursor.execute(sql.SQL(query).format(*[sql.Identifier(param) for param in params]))
        else:
            cursor.execute(query)

        try:
            # Fetch results if any (for SELECT queries)
            results = cursor.fetchall()
            return results
        except:
            print("No returns")
            # Commit changes (for INSERT, UPDATE, DELETE queries)
            connection.commit()    

    except (Exception, psycopg2.Error) as error:
        print(f"Error: {error}")
    finally:
        # Close the database connection
        if connection:
            cursor.close()
            connection.close()


*Select Example*

In [107]:
table_name = "users"

In [108]:
# Example query: Select all rows from a table
select_query = f"SELECT * FROM {table_name};"

# Execute the query
results = execute_query(select_query)

# Display the results
for row in results:
    print(row)

(1, 'john_doe', 'john.doe@example.com', datetime.datetime(2024, 1, 30, 3, 28, 29, 177083))
(2, 'jane_smith', 'jane.smith@example.com', datetime.datetime(2024, 1, 30, 3, 28, 29, 177083))
(3, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))
(4, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))
(6, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))
(7, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))
(9, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))
(10, 'john_doe', 'john.doe@example.com', datetime.datetime(2022, 1, 30, 12, 0))


*Insert Example*

In [109]:
# Example data to be inserted
insert_params = ('john_doe', 'john.doe@example.com', '2022-01-30 12:00:00')

# Example INSERT query
insert_query = f"""
    INSERT INTO {table_name} (username, email, created_at)
    VALUES {str(insert_params)}
"""

# Call the execute_query function to execute the INSERT query
execute_query(insert_query)

No returns
